In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import norm
from tqdm import tqdm 

In [51]:
df = pd.read_csv("features.csv")
df 

,id,eye,stimuli,color,glaucoma,mean_window_50_0,std_window_50_0,median_window_50_0,max_window_50_0,min_window_50_0,...,std_window_50_54,median_window_50_54,max_window_50_54,min_window_50_54,range_window_50_54,IQR_window_50_54,Q1_window_50_54,Q3_window_50_54,kurtosis_window_50_54,skewness_window_50_54
0,C001,left,ramp,white,no,32.233702,21.220448,29.165282,60.572493,0.000000,...,17.153265,55.027706,68.376038,0.000000,68.376038,1.782344,53.771554,55.553898,1.251011,-1.684972
1,C001,left,ramp,blue,no,54.801834,7.457179,56.462238,57.416459,13.402000,...,6.083763,54.958326,56.582669,23.579367,33.003302,1.043700,54.714285,55.757985,19.538668,-4.602760
2,C001,left,ramp,green,no,55.186039,5.784269,55.880359,68.521255,20.023360,...,5.171902,56.066412,61.083622,26.585171,34.498452,0.714797,55.668923,56.383720,18.591960,-4.266838
3,C001,left,flash,white,no,50.753054,15.557146,56.509845,80.000000,11.077743,...,6.081288,56.265699,59.186466,13.580236,45.606230,0.775210,55.944968,56.720178,41.859989,-6.504073
4,C001,left,flash,blue,no,47.545593,15.580917,55.304546,80.000000,10.388611,...,15.036102,55.169589,80.000000,12.908037,67.091963,2.188269,54.205255,56.393523,1.130352,-1.465107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,S020,right,flash,red,yes,48.162405,10.844362,51.866347,62.190289,11.731321,...,2.518823,50.626490,53.421669,40.842110,12.579559,1.261606,50.382777,51.644383,7.504337,-2.672959
652,S020,right,ramp,red,yes,51.051313,2.988132,51.567226,55.032780,37.716240,...,2.742470,49.961193,51.834238,37.091038,14.743200,2.602026,48.123849,50.725875,7.535957,-2.569689
653,S020,right,flash,white,yes,51.057405,7.632710,52.392627,53.833977,0.000000,...,16.909001,51.779498,58.379410,0.000000,58.379410,11.478749,41.723887,53.202636,1.894031,-1.819491
654,S020,right,ramp,green,yes,26.944068,23.689560,36.986803,54.678856,0.000000,...,0.646717,49.818981,50.948158,48.174379,2.773779,1.128346,49.218899,50.347245,-0.775603,-0.194488


In [37]:
df.dtypes

id                        object
eye                       object
stimuli                   object
color                     object
glaucoma                  object
                          ...   
IQR_window_50_54         float64
Q1_window_50_54          float64
Q3_window_50_54          float64
kurtosis_window_50_54    float64
skewness_window_50_54    float64
Length: 610, dtype: object

In [39]:
numeric = df.select_dtypes(["float64"]).columns
categorical = df.drop('id', axis = 1).select_dtypes(["object"]).columns

In [40]:
print(categorical)

Index(['eye', 'stimuli', 'color', 'glaucoma'], dtype='object')


In [41]:
print(numeric)

Index(['mean_window_50_0', 'std_window_50_0', 'median_window_50_0',
       'max_window_50_0', 'min_window_50_0', 'range_window_50_0',
       'IQR_window_50_0', 'Q1_window_50_0', 'Q3_window_50_0',
       'kurtosis_window_50_0',
       ...
       'std_window_50_54', 'median_window_50_54', 'max_window_50_54',
       'min_window_50_54', 'range_window_50_54', 'IQR_window_50_54',
       'Q1_window_50_54', 'Q3_window_50_54', 'kurtosis_window_50_54',
       'skewness_window_50_54'],
      dtype='object', length=605)


In [42]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

n_cols = 2

fig = make_subplots(
    rows=len(categorical) // n_cols , cols=n_cols, subplot_titles=categorical
)

# para todas las variables categóricas, graficar count de categorias
for idx, col in enumerate(categorical):
    row_idx = idx // n_cols + 1
    col_idx = idx % n_cols + 1

    data = df.loc[:, col]
    hist = go.Histogram(x=data, name=col)
    fig.add_trace(hist, row=row_idx, col=col_idx)

fig.update_layout(
    height=1200,
    title_text="Análisis Univariado de las Variables Categoricas",
    showlegend=False,
)
fig.show()

In [32]:
correlaciones = numeric.corr(method = 'pearson') # por default entrega correlacion de pearson
correlaciones

AttributeError: 'Index' object has no attribute 'corr'

In [43]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd

# Pipelines para columnas numéricas y categóricas (VarianceThreshold: elimina columnas con cero variabilidad)
numeric_transformations = Pipeline([
    ('scaler', MinMaxScaler()),
    #('variance_threshold', VarianceThreshold(threshold=0)) 
])

categoric_transformations = Pipeline([
    ('category_one_hot', OneHotEncoder(sparse_output=False, handle_unknown="ignore")),
    #('variance_threshold', VarianceThreshold(threshold=0))
])

# Preprocessor
preprocessor = ColumnTransformer(transformers=[
    ('numerical', numeric_transformations, numeric),
    ('categorical', categoric_transformations, categorical)
])

# Aplicar el preprocesamiento
df_pro = preprocessor.fit_transform(df)

# Recuperar los nombres de las nuevas columnas
ohe_feature_names = preprocessor.named_transformers_['categorical'].named_steps['category_one_hot'].get_feature_names_out(categorical)
final_columns = numeric + list(ohe_feature_names)

# Dado que VarianceThreshold puede eliminar algunas características, necesitamos actualizar los nombres de las columnas
# Primero, debemos verificar cuántas características fueron eliminadas por VarianceThreshold
num_features_after_numeric_threshold = preprocessor.named_transformers_['numerical'].named_steps['variance_threshold'].get_support(indices=True)
num_features_after_categorical_threshold = preprocessor.named_transformers_['categorical'].named_steps['variance_threshold'].get_support(indices=True)

# Filtrar las columnas finales
final_columns_numeric = [numeric[i] for i in num_features_after_numeric_threshold]
final_columns_categorical = [ohe_feature_names[i] for i in num_features_after_categorical_threshold]
final_columns = final_columns_numeric + final_columns_categorical

# Convertir el resultado a un DataFrame
df_pro = pd.DataFrame(df_pro, columns=final_columns)

ValueError: operands could not be broadcast together with shapes (605,) (10,) 

In [57]:
pd.set_option('future.no_silent_downcasting', True)
df = df.replace({'yes': 1, 'no': 0})

In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Supongamos que tienes un dataset cargado
# dataset = pd.read_csv('path_to_your_dataset.csv')

# Primero, obtener los ids únicos
unique_ids = df['id'].unique()

# Realizar el train_test_split a nivel de id
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42)

# Filtrar el df original usando los ids seleccionados
train_set = df[df['id'].isin(train_ids)]
test_set = df[df['id'].isin(test_ids)]

# Verifica los tamaños de los conjuntos
print(f"Tamaño del conjunto de entrenamiento: {len(train_set)}")
print(f"Tamaño del conjunto de prueba: {len(test_set)}")

Tamaño del conjunto de entrenamiento: 512
Tamaño del conjunto de prueba: 144


In [64]:
train_labels = train_set.loc[:, 'glaucoma']
test_labels = test_set.loc[:, 'glaucoma']
labels = df.loc[:, 'glaucoma']

In [63]:
print(train_labels)
print(test_labels)

0      0
1      0
2      0
3      0
4      0
      ..
619    1
620    1
621    1
622    1
623    1
Name: glaucoma, Length: 512, dtype: int64
64     0
65     0
66     0
67     0
68     0
      ..
651    1
652    1
653    1
654    1
655    1
Name: glaucoma, Length: 144, dtype: int64


In [65]:
# distribución original
labels.value_counts() / labels.count() * 100

glaucoma
0    51.219512
1    48.780488
Name: count, dtype: float64

In [66]:
# conjunto de entrenamiento
train_labels.value_counts() / train_labels.count() * 100

glaucoma
0    53.125
1    46.875
Name: count, dtype: float64

In [67]:
# conjunto de pruebas
test_labels.value_counts() / test_labels.count() * 100

glaucoma
1    55.555556
0    44.444444
Name: count, dtype: float64